In [1]:
import requests
from bs4 import BeautifulSoup
import pyodbc
import re
import time
import random


In [2]:
# Tạo session, tự động giữ cookie
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115 Safari/537.36"
})


In [3]:
# thay đổi các thông tin dưới đây cho đúng
server = "localhost"
database = "WH_market"
username = "sa"
password = "Ewddochin2202"

conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};DATABASE={database};UID={username};PWD={password}"
)
cursor = conn.cursor()
BASE = "https://monngonmoingay.com"

In [4]:
def safe_get(url, retries=3, backoff=1.0, timeout=10):
    """GET với retry khi lỗi"""
    for attempt in range(retries):
        try:
            r = session.get(url, timeout=timeout)
            if r.status_code == 200:
                return r.text
            else:
                print(f"HTTP {r.status_code} khi GET {url}")
        except Exception as e:
            print("Lỗi request:", e)
        time.sleep(backoff * (attempt + 1) + random.uniform(0, 0.5))
    return None



In [5]:
def parse_list_page(html, page_number):
    """
    Link, Tên, Ảnh, Trang, Khẩu phần, Độ khó, Thời gian
    """
    soup = BeautifulSoup(html, "html.parser")
    results = []

    # container
    container = soup.select_one(".relative.has-\\[\\.slider\\]:-mx-4.group > div") 

    if not container:
        print("Không tìm thấy container")
        return results

    # Duyệt item
    for item_div in container.find_all("div", recursive=False):
        a_tag = item_div.find("a", title=True)
        if not a_tag:
            continue

        link = BASE + a_tag.get("href", "") if a_tag.get("href").startswith("/") else a_tag.get("href", "")
        title = a_tag.get("title", "").strip()
        if not title:  
            continue

        # Hình ảnh
        img_tag = a_tag.find("img")
        img_url = BASE + img_tag.get("src", "") if img_tag and img_tag.get("src") else None
       
        tags_div = item_div.find("div", class_="tags")

        if tags_div:
            tag_children = tags_div.find_all("div")
            khau_phan = tag_children[0].get_text(strip=True) if len(tag_children) >= 1 else None
            do_kho = tag_children[1].get_text(strip=True) if len(tag_children) >= 2 else None
            thoi_gian = tag_children[2].get_text(strip=True) if len(tag_children) >= 3 else None
        else:
            khau_phan = do_kho = thoi_gian = None

        if link and title:
            results.append({
                "Link": link.strip(),
                "Ten": title.strip(),
                "Anh": img_url,
                "Trang": page_number,
                "Khẩu phần": khau_phan,
                "Độ khó": do_kho,
                "Thời gian thực hiện": thoi_gian
            })

    return results



In [6]:
def save_to_sql(data):
    for row in data:
        try:
            cursor.execute("""
                INSERT INTO update_mon
                ([Link món ăn], [Tên món ăn], [Hình ảnh], [Trang], [Khẩu phần], [Độ khó], [Thời gian thực hiện])
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, 
            row.get("Link"),
            row.get("Ten"),
            row.get("Anh"),
            row.get("Trang"),
            row.get("Khẩu phần"),
            row.get("Độ khó"),
            row.get("Thời gian thực hiện")
            )
            conn.commit()
        except Exception as e:
            print("Lỗi chèn:", e, row.get("Link"))
            


In [7]:
TOTAL_PAGES = 203  # tổng số trang

for page in range(1, TOTAL_PAGES + 1):
    url = f"{BASE}/tim-kiem-mon-ngon/page/{page}/"
    html = safe_get(url)
    
    if html:
        data = parse_list_page(html, page)
        if data:
            save_to_sql(data)
            print(f"Đã lưu {len(data)} món từ trang {page}")
        else:
            print(f"Trang {page} không có món ăn hoặc lỗi parse")
    else:
        print(f"Lỗi tải trang {page}")
    
    # sleep ngắn ngẫu nhiên 0.5 - 2 giây để tránh bị block
    time.sleep(random.uniform(0.5, 2.0))


SelectorSyntaxError: ':-mx-4' was detected as a pseudo-class and is either unsupported or invalid. If the syntax was not intended to be recognized as a pseudo-class, please escape the colon.
  line 1:
.relative.has-\[\.slider\]:-mx-4.group > div
                          ^